In [31]:
import requests
import datetime
import time
import pandas as pd
from pathlib import Path



In [32]:
def download_csv(url, output_file):
    """
    Downloads a CSV file from the given URL and saves it to the specified file.
    
    :param url: URL to download the CSV data from.
    :param output_file: Path to the local file where the CSV will be saved.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Ensure we notice bad responses

        # Write the content (CSV data) to a file in binary mode
        with open(output_file, 'wb') as f:
            f.write(response.content)
        
        print(f"CSV file has been successfully downloaded and saved as '{output_file}'.")
        
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")

In [33]:
raw_data_folder = "..\\data\\raw\\"
# download the raw data from national gas data portal
def download_raw_data():
    with open("..\\PUB ids.txt") as f:
        pubIds = f.read()
        pubIds = pubIds.replace("\n", ",").strip() 
    
    earliest = datetime.date(2020,4,1) # Download data going back 5 years
    # Loop from week 0 (today) to week 13 (13 weeks ago)
    download_from = datetime.date.today().replace(day=1) # start first download on first day of current month
    download_to = datetime.date.today() # end first download on today's date
    while(download_from > earliest):
        
        # Format the date in yyyy-mm-dd format
        formatted_from = download_from.strftime("%Y-%m-%d")
        formatted_to = download_to.strftime("%Y-%m-%d")
            
        csv_url = f"https://data.nationalgas.com/api/find-gas-data-download?applicableFor=Y&dateFrom={formatted_from}&dateTo={formatted_to}&dateType=GASDAY&latestFlag=Y&ids={pubIds}&type=CSV"
        month_format = download_from.strftime("%Y-%m")
        output_filename = f"{raw_data_folder}{month_format}.csv"

        download_csv(csv_url, output_filename)
        time.sleep(3) # brief courtesy sleep
        download_to = download_from - datetime.timedelta(days=1) # next download should go up to the day before the previous download start date
        download_from = download_to.replace(day=1) # next download should start on the first day of the month

In [49]:

def pivot(df, cols):

    #only keep the values we are interested in
    mask = df["Data Item"].isin(cols)

    df_filtered = df[mask]  

    # if there are duplicates for the field and gas day, take the latest
    df_latest = (
        df_filtered
        .sort_values("Applicable At")
        .groupby(["Gas Day", "Data Item"])
        .last()  # this takes the row with the highest (i.e. latest) "Applicable At" per group
        .reset_index()
    )
    
    # pivot to get 1 row per gas day
    df_latest = df_latest.pivot(index="Gas Day", columns="Data Item", values="Value").reset_index()
    
    # Drop 1 column that accounts for most of the NaNs
    df_latest.drop(columns=["Composite Weather Variable - Actual"], inplace=True)

    return df_latest

def load_data():
    #Read raw CSVs
    pathlist = list(Path(raw_data_folder).rglob('*.csv'))
    file_count = len(pathlist)
    dfs = []
    files_done = 0
    for path_obj in pathlist:
        path = str(path_obj)   
        
        df = pd.read_csv(path,
            parse_dates=["Applicable At", "Applicable For", "Generated Time"],
            dayfirst=True)

        df.rename(columns={'Applicable For': 'Gas Day'}, inplace=True)
        df['Gas Day'] = pd.to_datetime(df['Gas Day'], dayfirst=True)
        # daily summary columns: 

        daily_cols = df["Data Item"].unique()
        # print(daily_cols)
        # Get price and demand columns, to use as tommorow's ground truth, and with 1-3 days lag

        #label_cols = ["SAP, Actual Day", "SMP Buy, Actual Day", "SMP Sell, Actual Day", "Demand Actual, NTS, D+1"]

        #df_labels = pivot(df, label_cols)
        

        df_daily = pivot(df, daily_cols)
        dfs.append(df_daily)
        
        files_done += 1
        if files_done % 10 == 0:
            print(f"Processed {files_done} of {file_count} raw files")

    df = pd.concat(dfs)

    #Rename the columns that are going to be reused for ground truth and time series
    df.rename(columns={"SAP, Actual Day": 'Actual SAP', "SMP Buy, Actual Day": 'Actual SMPBuy', "SMP Sell, Actual Day": 'Actual SMPSell', "Demand Actual, NTS, D+1":"Actual demand"}, inplace=True)
    label_cols = ["Actual SAP", "Actual SMPBuy", "Actual SMPSell", "Actual demand"]

    # add lagged features
    lag_days = 5
    for i in range(1, lag_days+1):
        for col in label_cols:
            df[f"{col} D-{i}"] = df[col].shift(i)

    # Add labels for next day's actuals
    for col in label_cols:
        df[f"Next Day {col}"] = df[col].shift(-1)

    # There should be very few rows that have any NaNs so we can drop any that do
    df.dropna(inplace=True)
    return df

def split_train_test(df, split_date):
    """
    Splits the DataFrame into training and testing sets based on the split date.
    
    :param df: The DataFrame to split.
    :param split_date: The date to split the DataFrame on.
    :return: Tuple of (training set, testing set).
    """
    # Ensure that the 'Gas Day' column is in datetime format
    #df['Gas Day'] = pd.to_datetime(df['Gas Day'])

    # Split the DataFrame into training and testing sets
    train_df = df[df['Gas Day'] < split_date]
    test_df = df[df['Gas Day'] >= split_date]

    return train_df, test_df

In [53]:
#download_raw_data()
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)
df = load_data()

train, test = split_train_test(df, '2024-04-01')

train.head()

test.head()

Processed 10 of 40 raw files
Processed 20 of 40 raw files
Processed 30 of 40 raw files
Processed 40 of 40 raw files


Data Item,Gas Day,Aggregate LNG Importations - Daily Flow,Beach Including Norway - Daily Flow,Beach and IOG - Beach Delivery,Beach and IOG - Daily Flow,Composite Weather Variable - Cold,Composite Weather Variable - Normal,Composite Weather Variable - Warm,Demand - Cold,"Demand - Cold, (excluding interconnector and storage)",...,Actual SMPSell D-4,Actual demand D-4,Actual SAP D-5,Actual SMPBuy D-5,Actual SMPSell D-5,Actual demand D-5,Next Day Actual SAP,Next Day Actual SMPBuy,Next Day Actual SMPSell,Next Day Actual demand
0,2024-04-01,7.37140,182.25617,184.41427,184.41427,4.05,8.02,11.81,299.472882,260.912752,...,2.2306,207.595,2.3624,2.4399,2.2849,204.042,2.2736,2.3511,2.1633,198.632
1,2024-04-02,8.51758,184.04497,187.41845,187.41845,4.16,8.08,11.92,300.886855,260.524839,...,2.2484,192.069,2.3081,2.3856,2.2306,207.595,2.1472,2.2247,2.0697,189.183
2,2024-04-03,7.37140,178.28651,180.59271,180.59271,4.26,8.14,12.02,301.186013,258.769801,...,2.1613,181.574,2.3259,2.4034,2.2484,192.069,2.1333,2.2108,2.0558,197.660
3,2024-04-04,7.92970,185.05855,187.91865,187.91865,4.36,8.23,12.12,299.538190,257.080091,...,2.1864,193.899,2.2388,2.3163,2.1613,181.574,2.1673,2.2448,2.0898,189.113
4,2024-04-05,7.36850,182.57938,184.18328,184.18328,4.46,8.33,12.21,300.067806,255.535512,...,2.2621,195.872,2.2639,2.3414,2.1864,193.899,2.1956,2.2731,2.1181,186.450
